In [2]:
import pandas as pd
from tqdm import tqdm
# import vaex
# df=vaex.from_csv('/srv/abacus-3/Ethereum_token_txs_data/uniswapv2/uni-v2-pairs_with_tokens_decimal.csv',convert=True, chunk_size=5_000_000)
def read_large_csv(file_path, chunk_size=10000000):
    accumulated_data = pd.DataFrame()
    chunks = pd.read_csv(file_path, chunksize=chunk_size)
    for chunk in tqdm(chunks):
        accumulated_data = pd.concat([accumulated_data, chunk])
    return accumulated_data

In [3]:
daily_validator_index_data=read_large_csv('/local/scratch/exported/Ethereum_token_txs_data_TY_23/rewards/eth2_reward_ether/daily_attestation_reward/total_daily_attestation_rewards_sum.csv')

30it [03:08,  6.29s/it]


In [9]:
daily_validator_index_data.head(2)

,date,validator_index,total_reward
0,2022-09-15,0,2332229.0
1,2022-09-15,1,2300439.0


In [5]:
label_data=pd.read_csv('/local/scratch/exported/Ethereum_token_txs_data_TY_23/rewards/mining_staking/staking/dune_depositor_entity_data/validator_index_deposit_address_entity.csv')

/tmp/ipykernel_53532/3213145281.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  label_data=pd.read_csv('/local/scratch/exported/Ethereum_token_txs_data_TY_23/rewards/mining_staking/staking/dune_depositor_entity_data/validator_index_deposit_address_entity.csv')


In [6]:
label_data.head(2)

,test,block_number,amount_staked,depositor_address,entity,entity_unique_name,entity_category,tx_hash,deposit_index,withdrawal_address,evt_index
0,11185311_1.00000_0x798cf8fc6f212da30b10cac6e05...,11185311,1.0,0x798cf8fc6f212da30b10cac6e05b4bf275c34bff,NaN,NaN,NaN,0x7085c586686d666e8bb6e9477a0f0b09565b2060a11f...,0,NaN,131
1,11191448_1.00000_0xc34eb7e3f34e54646d7cd140bb7...,11191448,1.0,0xc34eb7e3f34e54646d7cd140bb7c20a466b3e852,NaN,NaN,NaN,0xa90ed27521c07e66d52db6ee47d729d1182299253037...,1,NaN,126


In [8]:
label_data.rename(columns={'deposit_index':'validator_index'},inplace=True)

## merge index_address_pool

In [11]:
label_data1=label_data[['validator_index','depositor_address','entity','entity_unique_name','entity_category']]

In [14]:
daily_validator_index_data1=daily_validator_index_data.merge(label_data1, on='validator_index',how='left')

In [16]:
daily_validator_index_data1.shape

(299511488, 7)

In [25]:
daily_validator_index_data1.to_csv('/local/scratch/exported/Ethereum_token_txs_data_TY_23/rewards/eth2_reward_ether/daily_attestation_reward/total_daily_attestation_rewards_sum_with_entity.csv',index=False)

## cluster in the address level

In [19]:
daily_validator_index_data1.head(2)

,date,validator_index,total_reward,depositor_address,entity,entity_unique_name,entity_category
0,2022-09-15,0,2332229.0,0x798cf8fc6f212da30b10cac6e05b4bf275c34bff,NaN,NaN,NaN
1,2022-09-15,1,2300439.0,0xc34eb7e3f34e54646d7cd140bb7c20a466b3e852,NaN,NaN,NaN


In [20]:
#统计entity列不为空的行数
daily_validator_index_data1[daily_validator_index_data1['entity'].notnull()].shape

(255662959, 7)

In [21]:
daily_validator_index_data2=daily_validator_index_data1.groupby(['depositor_address','date'])[['total_reward']].sum().reset_index()

In [22]:
daily_validator_index_data2

,depositor_address,date,total_reward
0,0x00000000006267dd559d14f5adef3f777ae7bb0e,2023-01-28,388087.0
1,0x00000000006267dd559d14f5adef3f777ae7bb0e,2023-01-29,3003823.0
2,0x00000000006267dd559d14f5adef3f777ae7bb0e,2023-01-30,3008072.0
3,0x00000000006267dd559d14f5adef3f777ae7bb0e,2023-01-31,4681225.0
4,0x00000000006267dd559d14f5adef3f777ae7bb0e,2023-02-01,5982929.0
...,...,...,...
49170560,0xfffffffff0cee49cff5550d64455d52e1e20824d,2023-12-28,2264815.0
49170561,0xfffffffff0cee49cff5550d64455d52e1e20824d,2023-12-29,2247858.0
49170562,0xfffffffff0cee49cff5550d64455d52e1e20824d,2023-12-30,2236460.0
49170563,0xfffffffff0cee49cff5550d64455d52e1e20824d,2023-12-31,2241276.0


In [26]:
daily_validator_index_data2.to_csv('/local/scratch/exported/Ethereum_token_txs_data_TY_23/rewards/eth2_reward_ether/daily_attestation_reward/total_daily_attestation_rewards_sum__address_level.csv',index=False)

## cluster in the staking pool level

In [23]:
daily_validator_index_data3=daily_validator_index_data1.groupby(['entity','date'])[['total_reward']].sum().reset_index()

In [24]:
daily_validator_index_data3

,entity,date,total_reward
0,04221.eth,2022-12-02,2279176.0
1,04221.eth,2022-12-03,3056870.0
2,04221.eth,2022-12-04,3021322.0
3,04221.eth,2022-12-05,3058092.0
4,04221.eth,2022-12-06,3056658.0
...,...,...,...
589549,📤📤📤.eth,2023-12-28,11249686.0
589550,📤📤📤.eth,2023-12-29,11200908.0
589551,📤📤📤.eth,2023-12-30,11126152.0
589552,📤📤📤.eth,2023-12-31,11191297.0


In [27]:
daily_validator_index_data3.to_csv('/local/scratch/exported/Ethereum_token_txs_data_TY_23/rewards/eth2_reward_ether/daily_attestation_reward/total_daily_attestation_rewards_sum__stakingpool_level.csv',index=False)